This Notebook is for 3D holograms with target intensity specified on a rectangular grid (512x512x3 pixels). Here we use extensively trained models for 512x512x3 and 1024x1024x3.

**Step 1** - We import the necessary modules and we clone the Github repository into the working directory of this google colab session.

In [2]:
!pip install tensorflow==2.8.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.0/498.0 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 113.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 67.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  Attempting uninstall: tensorboard-data-server
    Found existing installati

In [3]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from time import time
!git clone https://github.com/vedang-04/Exploratory_Project
os.chdir('Exploratory_Project/DeepCGH')

Cloning into 'Exploratory_Project'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 44 (delta 6), reused 11 (delta 1), pack-reused 24
Receiving objects: 100% (44/44), 111.17 MiB | 23.23 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Updating files: 100% (24/24), done.


Now we can use `deepcgh.DeepCGH_Datasets` to generate the dataset and train a model with it.

`DeepCGH_Datasets`: is a module that synthesizes training data set.

`DeepCGH`: is the main module that contains the pre-trained model and performs the training.

In [4]:
from deepcgh import DeepCGH_Datasets, DeepCGH
from utils import GS3D, display_results, get_propagate

We specify the properties of the training dataset that will be used to train the DeepCGH model. Here we only create a small dataset (N=100) since creating and storing a large dataset is time-consuming.

In [5]:
retrain = True
coordinates = False

data = {'path' : 'DeepCGH_Datasets/Disks',# path to store the data
        'shape' : (512, 512, 3),# shape of the holograms. The last dimension determines the number of depth planes
        'object_type' : 'Disk',# shape of the object in simulated images, can be disk, square, or line
        'object_size' : 10,# has no effect if object type is 'Line'
        'object_count' : [27, 48],# number of random objects to be created
        'intensity' : [0.2, 1],# the (range of) intensity of each object. If a range is specified, for each object the intensity is randomly determined
        'normalize' : True,# if the data is 3D, it'll normalize the intensities from plane to plane (see manuscript fot more info)
        'centralized' : False,# avoids putting objects near the edges of the hologram (useful for practical optogenetics applications)
        'N' : 100, # number of sample holograms to generate
        'train_ratio' : 90/100,# the ratio of N that will be used for training
        'compression' : 'GZIP',# tfrecords compression format
        'name' : 'target',# name of the dictionary that contains the targets (leave as "target" if you're not changing the structure of network input)
        }

We generate the dataset by calling the `getDataset` method:

In [6]:
dset = DeepCGH_Datasets(data)
dset.getDataset()

/content/Exploratory_Project/DeepCGH/deepcgh.py:72: UserWarning: File does not exist. New dataset will be generated once getDataset is called.
  warnings.warn('File does not exist. New dataset will be generated once getDataset is called.')


Current working directory is:
/content/Exploratory_Project/DeepCGH 

/content/Exploratory_Project/DeepCGH/DeepCGH_Datasets/Disks/Disk_SHP(512, 512, 3)_N100_SZ10_INT[0.2, 1]_Crowd[27, 48]_CNTFalse_Split.tfrecords
Generating data...


100%|██████████| 100/100 [00:13<00:00,  7.51it/s]


Here we define the model parameters:

In [7]:
model = {'path' : 'DeepCGH_Models/Disks',# the path to the saved model
        'plane_distance':0.005,# the physical distance between depth planes when we're doing 3D holography
         # physical setup parameters
        'wavelength':1e-6,# the wavelength of the laser (both simulations and experiments)
        'pixel_size':0.000015,# size of the SLM pixel sizes
        'int_factor':16,# the interleaving factor
         # CNN model and training parameters
        'n_kernels':[ 64, 128, 256],# the number of kernels in the U-Net model (see the manuscript)
        'input_name':'target',# name of the input layer in the U-Net model
        'output_name':'phi_slm',# name of the output layer
        'lr' : 1e-7,# learning rate of the optimizer
        'batch_size' : 8,
        'epochs' : 1,
        'shuffle' : 8,# determine how many samples are going to be shuffled
        'token' : '64',# string to be attached to the name of the model to differentiate it from similar models
        'max_steps' : 5000 , # maxmimum number of batches/steps to be processed
        'quantization': 2,
        'focal_point': 1,
        }


**Step 2** - We create the DeepCGH module and train it. The code includes extensively trained models for 512x512x3 and 1024x1024x3. For those models, training will begin from the pretrained model and run for one more epoch on the dataset we just generated. In other case,we have to train the model that takes time as it will begin from scratch.

In [8]:
dcgh = DeepCGH(data, model)
# Training
dcgh.train(dset)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Looking for trained models in:
/content/Exploratory_Project/DeepCGH 

Model already exists.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use standard file utilities to get mtimes.
Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead


**Step 3** - Once the model is trained/loaded, we can start inference. We generate a random sample:

In [9]:
image = dset.get_randSample()[np.newaxis,...]
# making inference is as simple as calling the get_hologram method
phase = dcgh.get_hologram(np.zeros_like(image)) # the very first inference takes a long time (a known tensorflow characteristic)

We compute the phase and measure the inference time:

In [10]:
t0 = time()
phase = dcgh.get_hologram(image)
t = time() - t0

Now we simulate the hologram that this phase will generate:

In [19]:
# reconstruction = get_propagate(data, model)(phase)
# display_results(image, phase, reconstruction, t)

In [16]:
phase

array([[[[ 1.0471976],
         [ 3.1415927],
         [-1.0471976],
         ...,
         [-3.1415927],
         [ 1.0471976],
         [-3.1415927]],

        [[-3.1415927],
         [ 1.0471976],
         [-1.0471976],
         ...,
         [-3.1415927],
         [ 3.1415927],
         [-1.0471976]],

        [[-1.0471976],
         [-3.1415927],
         [ 1.0471976],
         ...,
         [ 1.0471976],
         [-3.1415927],
         [ 1.0471976]],

        ...,

        [[-1.0471976],
         [ 1.0471976],
         [-1.0471976],
         ...,
         [ 1.0471976],
         [ 1.0471976],
         [ 3.1415927]],

        [[ 1.0471976],
         [-1.0471976],
         [ 3.1415927],
         ...,
         [ 1.0471976],
         [-1.0471976],
         [ 3.1415927]],

        [[ 3.1415927],
         [ 1.0471976],
         [-3.1415927],
         ...,
         [-3.1415927],
         [ 1.0471976],
         [-1.0471976]]]], dtype=float32)

In [17]:
image

array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]])